# Parte 1: Estadísticas para ingenieros que miran el cielo


## 1.1 Promedio y desvio

### Melbourne

In [ ]:
import csv
import statistics
archivo_melbourne='temperature_Melbourne_celsius.csv'
valores= []
promedio_melbourne = 0.0
with open(archivo_melbourne, 'r') as archivo:
    reader = csv.reader(archivo)
    next(reader)  
    valores = [float(fila[0]) for fila in reader]

promedio = sum(valores) / len(valores)
varianza = sum((x - promedio)**2 for x in valores ) / (len(valores) - 1) 
desvio=varianza**-2

print(promedio)
print(varianza)
print(desvio)


### Oslo

In [ ]:
import csv
import statistics
archivo_melbourne='temperature_Oslo_celsius.csv'
valores= []
promedio_melbourne = 0.0
with open(archivo_melbourne, 'r') as archivo:
    reader = csv.reader(archivo)
    next(reader)  
    valores = [float(fila[0]) for fila in reader]

promedio = sum(valores) / len(valores)
varianza = sum((x - promedio)**2 for x in valores ) / (len(valores) - 1) 
desvio=varianza**-2

print(promedio)
print(varianza)
print(desvio)

### Quito

In [ ]:
import csv
import statistics
archivo_melbourne='temperature_Quito_celsius.csv'
valores= []
promedio_melbourne = 0.0
with open(archivo_melbourne, 'r') as archivo:
    reader = csv.reader(archivo)
    next(reader)  
    valores = [float(fila[0]) for fila in reader]

promedio = sum(valores) / len(valores)
varianza = sum((x - promedio)**2 for x in valores ) / (len(valores) - 1) 
desvio=varianza**-2

print(promedio)
print(varianza)
print(desvio)